# Setup Environment and Dependencies
Import required libraries (torch, transformers, sklearn) and set up initial configurations including rich console setup.

In [1]:
# Cell 1 - Setup and Imports
from model import *  # Import all functions from model.py
import torch
import numpy as np
import matplotlib.pyplot as plt
from rich.console import Console
from rich.traceback import install

# Initialize console and traceback
install(show_locals=False)
console = Console()

In [4]:
# Define Evidence dataclass
@dataclass
class Evidence:
    """Represents a piece of evidence associated with a claim."""
    evidence_id: str
    content: str
    embedding: Optional[np.ndarray] = None

# Define Claim dataclass
@dataclass
class Claim:
    """Represents a claim with its associated metadata and evidences."""
    claim_id: str
    content: str
    label: int
    explanation: str
    evidences: List[Evidence] = field(default_factory=list)
    embedding: Optional[np.ndarray] = None
    clustered_evidences: Optional[Dict[int, List[Evidence]]] = None

# Cell 2 - Load Example Data
# Load example data from LIAR dataset
example_data = json.load(open("../dataset/LIAR-RAW/test.json"))[403]
example_claim = Claim(
    claim_id="403",
    content=example_data["claim"],
    label=example_data["label"],
    explanation=example_data["explain"],
)

# Load evidences
for evidence in example_data["reports"]:
    example_claim.evidences.append(
        Evidence(evidence_id=evidence["report_id"], content=evidence["content"])
    )

print(f"Claim: {example_claim.content}")
print(f"Number of evidences: {len(example_claim.evidences)}")

Claim: Im not a conspiracy theorist and I never allow conspiracy theorists on my program.
Number of evidences: 28


In [6]:
evidences = example_claim.evidences

# Get evidence texts
evidence_texts = [ev.content for ev in evidences]

# Adjust parameters based on evidence count
batch_size = min(32, len(evidence_texts))

In [9]:
evidence_texts, len(evidence_texts), batch_size

(['In the current research , we investigate whether belief in conspiracy theory satisfy people ’ s need for uniqueness .\n We find that the tendency to believe in conspiracy theory be associate with the feeling of possess scarce information about the situation explain by the conspiracy theory ( Study 1 ) and high need for uniqueness ( Study 2 ) .\n Further two study use two different manipulation of need for uniqueness ( Studies 3 and 4 ) show that people in a high need for uniqueness condition display high conspiracy belief than people in a low need for uniqueness condition .\n These study suggest that conspiracy theory may serve people ’ s desire to be unique , highlight a motivational underpinning of conspiracy belief .\n , the 2001 tsunami in Japan ) , many people start to disbelieve official explanation in favor of what be know a conspiracy theory .\n For example , one conspiracy theory attribute the Charlie Hebdo attack to Mossad a an effort to make Muslims look bad .\n In recent

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TextUMC().to(device)

for i in range(0, len(evidence_texts), batch_size):
    batch_texts = evidence_texts[i : i + batch_size]
    embeddings = model(batch_texts)

print(embeddings.detach().cpu().numpy(), embeddings.shape)  

Output()

In [5]:
# Cell 3 - Model Training
# Initialize model and train
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TextUMC().to(device)

trained_evidences, metrics = umc_train(
    model=model,
    evidences=example_claim.evidences,
    num_clusters=min(len(example_claim.evidences), 5),
    batch_size=32,
    num_epochs=10,
    learning_rate=2e-5,
)

Output()

Output()

Output()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│    3 device = torch.device("cuda" if torch.cuda.is_available() else "cpu")                       │
│    4 model = TextUMC().to(device)                                                                │
│    5                                                                                             │
│ ❱  6 trained_evidences, metrics = umc_train(                                                     │
│    7 │   model=model,                                                                            │
│    8 │   evidences=example_claim.evidences,                                                      │
│    9 │   num_clusters=min(len(example_claim.evidences), 5),                                      │
│                                                                                                  │
│ /home/lonk/codes/thesis-test-code/TextUMC/model.py:430 in umc_train                              │
│                                                                                                  │
│   427 │   │   │   embeddings = model(batch_texts)                                                │
│   428 │   │   │                                                                                  │
│   429 │   │   │   # Get cluster assignments                                                      │
│ ❱ 430 │   │   │   _, pseudo_labels, current_centroids = cluster_and_select_samples(              │
│   431 │   │   │   │   embeddings, num_clusters, 0.5, current_centroids, model.device             │
│   432 │   │   │   )                                                                              │
│   433                                                                                            │
│                                                                                                  │
│ /home/lonk/codes/thesis-test-code/TextUMC/model.py:258 in cluster_and_select_samples             │
│                                                                                                  │
│   255 │   │   n_init=5,                                                                          │
│   256 │   │   max_iter=200,                                                                      │
│   257 │   )                                                                                      │
│ ❱ 258 │   kmeans.fit(embeddings.detach().cpu().numpy())                                          │
│   259 │   cluster_labels = kmeans.labels_                                                        │
│   260 │   new_centroids = kmeans.cluster_centers_                                                │
│   261                                                                                            │
│                                                                                                  │
│ /home/lonk/micromamba/envs/thesis/lib/python3.12/site-packages/sklearn/base.py:1389 in wrapper   │
│                                                                                                  │
│   1386 │   │   │   │   │   prefer_skip_nested_validation or global_skip_validation               │
│   1387 │   │   │   │   )                                                                         │
│   1388 │   │   │   ):                                                                            │
│ ❱ 1389 │   │   │   │   return fit_method(estimator, *args, **kwargs)                             │
│   1390 │   │                                                                                     │
│   1391 │   │   return wrapper                                                                    │
│   1392                                                     

# Training Pipeline
Implement the training loop with clustering-based pseudo-labels and contrastive learning objectives.

In [4]:
# Cell 4 - Clustering and Visualization
# Get evidence embeddings
evidence_texts = [ev.content for ev in example_claim.evidences]
with torch.no_grad():
    evidence_embeddings = model(evidence_texts)
    evidence_embeddings_np = evidence_embeddings.cpu().numpy()

# Cluster evidences
n_clusters = min(len(evidence_texts), 5)
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(evidence_embeddings_np)

# Print clusters
print_evidence_clusters(example_claim, cluster_labels)

# Group evidences
example_claim.clustered_evidences = {
    i: [ev for ev, label in zip(example_claim.evidences, cluster_labels) if label == i]
    for i in range(n_clusters)
}

# Visualize clusters
visualize_clusters(
    evidence_embeddings_np,
    cluster_labels,
    f"Evidence Clusters for Claim {example_claim.claim_id}",
    "evidence_clusters.png"
)

Output()

Output()

/home/lonk/micromamba/envs/thesis/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: 
Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,

/home/lonk/micromamba/envs/thesis/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid
value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)

/home/lonk/micromamba/envs/thesis/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: 
Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,

/home/lonk/micromamba/envs/thesis/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid
value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)

Output()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:216                                                                                  │
│                                                                                                  │
│   213 model = TextUMC().to(device)                                                               │
│   214                                                                                            │
│   215 # Train model                                                                              │
│ ❱ 216 trained_evidences, metrics = umc_train(                                                    │
│   217 │   model=model,                                                                           │
│   218 │   evidences=example_claim.evidences,                                                     │
│   219 │   num_clusters=min(len(example_claim.evidences), 5),                                     │
│                                                                                                  │
│ in umc_train:193                                                                                 │
│                                                                                                  │
│   190 │   │   │   embeddings = model(batch_texts)                                                │
│   191 │   │   │                                                                                  │
│   192 │   │   │   # Get cluster assignments                                                      │
│ ❱ 193 │   │   │   _, pseudo_labels, current_centroids = cluster_and_select_samples(              │
│   194 │   │   │   │   embeddings, num_clusters, 0.5, current_centroids, model.device             │
│   195 │   │   │   )                                                                              │
│   196                                                                                            │
│                                                                                                  │
│ in cluster_and_select_samples:21                                                                 │
│                                                                                                  │
│    18 │   │   n_init=5,                                                                          │
│    19 │   │   max_iter=200,                                                                      │
│    20 │   )                                                                                      │
│ ❱  21 │   kmeans.fit(embeddings.detach().cpu().numpy())                                          │
│    22 │   cluster_labels = kmeans.labels_                                                        │
│    23 │   new_centroids = kmeans.cluster_centers_                                                │
│    24                                                                                            │
│                                                                                                  │
│ /home/lonk/micromamba/envs/thesis/lib/python3.12/site-packages/sklearn/base.py:1389 in wrapper   │
│                                                                                                  │
│   1386 │   │   │   │   │   prefer_skip_nested_validation or global_skip_validation               │
│   1387 │   │   │   │   )                                                                         │
│   1388 │   │   │   ):                                                                            │
│ ❱ 1389 │   │   │   │   return fit_method(estimator, *args, **kwargs)                             │
│   1390 │   │                                                                                     │
│   1391 │   │   return wrapper                                                                    │
│   1392                                                     

# Clustering and Visualization
Implement functions for clustering embeddings, visualizing clusters with PCA, and displaying results using rich tables.

In [ ]:
# Clustering and Visualization

# Function to cluster claims
def cluster_claims(claims, num_clusters, random_state=42):
    """Clusters the claims using the trained model.

    Args:
        claims (List[Claim]): List of Claim objects.
        num_clusters (int): Number of clusters.

    Returns:
        List[Claim]: Claims with assigned cluster labels.
    """
    all_embeddings = np.array(
        [claim.embedding for claim in claims]
    )  # Extract all of the embeddings
    kmeans = KMeans(
        n_clusters=num_clusters, random_state=random_state, n_init=10
    )  # Perform K-Means using embeddings.
    kmeans.fit(all_embeddings)
    cluster_labels = kmeans.labels_  # Extract the cluster labels.

    for i, claim in enumerate(claims):
        claim_cluster = cluster_labels[i]
        if claim.clustered_evidences is None:
            claim.clustered_evidences = {}
        if claim_cluster not in claim.clustered_evidences:
            claim.clustered_evidences[claim_cluster] = []
        claim.clustered_evidences[claim_cluster].extend(
            claim.evidences
        )  # assign cluster to the evidences

    return claims

# Function to visualize clusters
def visualize_clusters(embeddings: np.ndarray, labels: np.ndarray, title: str, save_path: str):
    """Plot clusters using PCA for dimensionality reduction"""
    # Reduce to 2D for visualization
    pca = PCA(n_components=2)
    reduced_embeddings = pca.fit_transform(embeddings)

    # Create scatter plot
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(
        reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=labels, cmap="viridis"
    )
    plt.colorbar(scatter)
    plt.title(title)
    plt.xlabel("First Principal Component")
    plt.ylabel("Second Principal Component")
    plt.savefig(save_path)
    plt.close()

# Function to print evidence clusters
def print_evidence_clusters(claim: Claim, cluster_labels: np.ndarray):
    """Display evidence clusters in a readable format"""
    table = Table(title=f"Evidence Clusters for Claim: {claim.claim_id}")
    table.add_column("Cluster", justify="center", style="cyan")
    table.add_column("Evidence Content", justify="left", style="green")

    for cluster_id in range(max(cluster_labels) + 1):
        # Get evidences for this cluster
        cluster_evidences = [
            ev
            for ev, label in zip(claim.evidences, cluster_labels)
            if label == cluster_id
        ]

        # Add row for each evidence in cluster
        for idx, evidence in enumerate(cluster_evidences):
            # Truncate content for readability
            content = (
                evidence.content[:100] + "..."
                if len(evidence.content) > 100
                else evidence.content
            )
            table.add_row(f"Cluster {cluster_id}" if idx == 0 else "", content)
        # Add separator between clusters
        table.add_row("", "")

    console.print(table)

# Get evidence embeddings
evidence_texts = [ev.content for ev in example_claim.evidences]
with torch.no_grad():
    evidence_embeddings = model(evidence_texts)
    evidence_embeddings_np = evidence_embeddings.cpu().numpy()

# Cluster evidences
n_clusters = min(len(evidence_texts), 5)
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(evidence_embeddings_np)

# Print clusters
print_evidence_clusters(example_claim, cluster_labels)

# Group evidences
example_claim.clustered_evidences = {
    i: [
        ev
        for ev, label in zip(example_claim.evidences, cluster_labels)
        if label == i
    ]
    for i in range(n_clusters)
}

# Visualize clusters
plot_path = "evidence_clusters.png"
visualize_clusters(
    evidence_embeddings_np,
    cluster_labels,
    f"Evidence Clusters for Claim {example_claim.claim_id}",
    plot_path,
)